- 自定义神经网络 + 访问参数 + 自定义层 + 读写文件

# 1. 层和块

In [4]:
import torch
from torch import nn
from torch.nn import functional as F

# nn.Linear有默认初始化
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)    # 2是批量大小，20是输入维度，所以net线性层数输入为20
net(X)

tensor([[-0.1587,  0.1611,  0.0056,  0.4666, -0.0986, -0.0558,  0.2867, -0.0337,
         -0.1056, -0.1919],
        [-0.1198,  0.3089,  0.0037,  0.3942, -0.1108, -0.0517,  0.1984, -0.0118,
         -0.0220, -0.1338]], grad_fn=<AddmmBackward>)

### nn.Sequential定义了一种特殊的Module。在Pytorch里，module是一个很重要的概念

- 任何的层与神经网络都是module的一个子类

In [5]:
# 自定义块（自定义mlp实现上面的模型）

class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        super().__init__()    # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)     # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

# nn.ReLU是一个层结构（或者说是一个类），必须放在nn.Module中。F.relu是一个函数

### 实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层，实现与之前一样的函数

In [6]:
net = MLP()
net(X)

tensor([[ 0.1426, -0.0279,  0.1653,  0.0338,  0.1751,  0.0290, -0.0180,  0.0950,
         -0.2281,  0.1150],
        [ 0.1585, -0.0684,  0.2853,  0.1438,  0.2383,  0.0324, -0.0236, -0.0057,
         -0.1298,  0.1387]], grad_fn=<AddmmBackward>)

### 顺序块

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):    # *args: list of input argument
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0790,  0.2283,  0.1794, -0.3227, -0.2281, -0.1575,  0.1452, -0.1098,
          0.0490, -0.4712],
        [-0.1850,  0.1990,  0.0770, -0.3088, -0.1134, -0.3073,  0.1700, -0.1171,
          0.0890, -0.3641]], grad_fn=<AddmmBackward>)

### 当sequential类不能满足需求时，可以在init和forward里自定义函数
- 在正向传播函数中执行代码

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(0.2289, grad_fn=<SumBackward0>)

### 混合搭配各种组合块的方法

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0698, grad_fn=<SumBackward0>)

### 总结
- 一个块可以由许多层组成；一个块可以由许多块组成。
- 块可以包含代码。
- 块负责大量的内部处理，包括参数初始化和反向传播。
- 层和块的顺序连接由Sequential块处理。

# 2. 访问参数（参数管理）

In [10]:
# 首先关注具有单隐藏层的多层感知机

import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

# 之所以没有初始化w，是因为在调用net()的时候，PyTroch自动初始化了

tensor([[-0.3345],
        [-0.3280]], grad_fn=<AddmmBackward>)

### 访问每层的参数

In [11]:
# net[2]：返回nn.Sequential最后一层（线性层 nn.Linear(8, 1）
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1482,  0.0290,  0.2475,  0.3481, -0.1811, -0.1143,  0.1727, -0.0954]])), ('bias', tensor([-0.3471]))])


### 直接访问目标参数

In [12]:
print(type(net[2].bias))
# Parameter 定义一个可以优化的参数
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3471], requires_grad=True)
tensor([-0.3471])


In [13]:
net[2].weight.data

tensor([[ 0.1482,  0.0290,  0.2475,  0.3481, -0.1811, -0.1143,  0.1727, -0.0954]])

In [14]:
# 通过grad访问梯度
net[2].weight.grad == None
# 因为没有做反向计算，所以grad=none。当backward后（训练后），grad就能访问元素

True

## 一次性访问所有参数

In [15]:
# *：解包元组
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
# 0是第1线性层，2是第二线性（全连接）层。1是relu层，拿不出来

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [16]:
print([(name, param.shape) for name, param in net[0].named_parameters()])
print([(name, param.shape) for name, param in net.named_parameters()])

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
[('0.weight', torch.Size([8, 4])), ('0.bias', torch.Size([8])), ('2.weight', torch.Size([1, 8])), ('2.bias', torch.Size([1]))]


In [17]:
# 通过名字获取参数
net.state_dict()['2.bias'].data

tensor([-0.3471])

### 从嵌套块收集参数

In [18]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    # block2嵌套4个block1
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())    # f'block {i}'传一个字符串的名字(功能一样)，而非返回01234...
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3055],
        [-0.3052]], grad_fn=<AddmmBackward>)

## 显示设计的网络结构

In [19]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


### 因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。 下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。

In [20]:
rgnet[0][1][0].bias.data

tensor([ 0.4108,  0.2007,  0.3326,  0.2777,  0.4136, -0.0349,  0.2626,  0.4421])

# 内置初始化

In [21]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)    # 下划线表示会读输入进行改变
        nn.init.zeros_(m.bias)

# apply方法对net所有的layers都进行内置初始化
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0010,  0.0007,  0.0172, -0.0120]), tensor(0.))

In [22]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)    # 把权重初始化成值为1
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

## 对某些块应用不同的初始化方法

In [23]:
def xavier(m):    # xavier初始化：每一层输出的方差应该尽量相等
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.4617,  0.2949, -0.5163, -0.5965])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化

<img src="./pic/自定义初始化.PNG" width=500 height=500>

In [25]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 保留绝对值大于5的权重，否则设成0
        m.weight.data *= m.weight.data.abs() >= 5    # >=的优先级高于*=
        # 这里*=的代码相当于先计算一个布尔矩阵，然后再用布尔矩阵的对应元素去乘以原始矩阵的每个元素

        net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-7.0534, -0.0000, -9.8683, -9.1118],
        [-0.0000,  0.0000, -6.1340, -9.7358]], grad_fn=<SliceBackward>)

### 可以直接设置参数

In [27]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -8.8683, -8.1118])

### 参数绑定（参数共享）
- 有时我们希望在多个层间共享参数： 我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

In [28]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),    # 第0、6个隐藏层权重是自己的
                    shared, nn.ReLU(),   # 第2、4个隐藏层share权重
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)

# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])

# 如果第二层权重改变，第4层也会一起改变。因为指向同一个parameter实例
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 3. 自定义层
- 自定义层与自定义网络没区别，因为层也是`nn.module`的子类

In [6]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):    # 子类不定义init就会调用父类init
        super().__init__()

    def forward(self, X):
        return X - X.mean()
    

layer = CenteredLayer()
X = torch.FloatTensor([1, 2, 3, 4, 5])
layer(X)

tensor([-2., -1.,  0.,  1.,  2.])

In [9]:
X.mean(), layer(X).mean()

(tensor(3.), tensor(0.))

## 将层作为组件合并到更复杂的模型中

In [5]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

# 作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
# 由于处理的是浮点数，因为存储精度的原因，仍然可能会看到一个非常小的非零数

tensor(-3.2596e-09, grad_fn=<MeanBackward0>)

## 带参数的层

- 调用参数类：`nn.Parameter`
- 定义一个代参数（W,b）的自定义层
    - 通过nn.Parameter包裹住初始化的值

In [18]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))  # 自动更新原权重
        self.bias = nn.Parameter(torch.randn(units,))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    

linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-1.0041,  0.0068, -1.1608],
        [ 0.4268, -0.0303,  0.4747],
        [ 0.4850,  0.3256, -0.6011],
        [ 0.6265, -0.3294, -0.2187],
        [ 0.3967,  1.1555, -1.7935]], requires_grad=True)

In [13]:
# torch.rand和torch.randn
# rand是从0-1的均匀分布中随机抽样，randn是从0-1的正态分布中抽样，所以.mean()不为0
p = torch.rand(2, 3)
q = torch.randn(2, 3)
p, q, p.mean(), q.mean()

(tensor([[0.9828, 0.9877, 0.6760],
         [0.1421, 0.6556, 0.1884]]),
 tensor([[-0.6655, -1.9394,  0.4634],
         [-0.8074,  0.7427, -0.0662]]),
 tensor(0.6054),
 tensor(-0.3787))

### 使用自定义层直接执行前向传播计算

In [24]:
linear(torch.rand(2, 5))

tensor([[0.4066, 0.0000, 0.0000],
        [0.7730, 0.9556, 0.0000]])

### 使用自定义层构建模型

In [29]:
# 使用自定义实例放入sequential里，参与网络构造

net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[6.2110],
        [0.0000]])

# 4. 读写文件
- 保存`torch.save`和加载`torch.load`张量

In [30]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

## 存储一个张量列表，然后把它们读回内存

In [31]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

## 写入或读取从字符串映射到张量的字典
- 当要读取或写入模型中的所有权重时，这很方便

In [32]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 加载和保存模型参数

In [33]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

- 将模型的参数存储在一个叫做`mlp.params`的文件

In [35]:
torch.save(net.state_dict(), 'mlp.params')

- 实例化了原始多层感知机模型的一个备份。 这里我们不需要随机初始化模型参数，而是直接读取文件中存储的参数。

In [37]:
clone = MLP()    # 先声明MLP（生成网络，参数已随机初始化）
clone.load_state_dict(torch.load('mlp.params'))    # 通过load来覆盖随机初始化的权重
clone.eval()    # 显示网络结构

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

- 验证：
    - 由于两个实例具有相同的模型参数，在输入相同的X时， 两个实例的计算结果应该相同

In [38]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

train模式（net.train())和eval模式（net.eval())。
一般的神经网络中，这两种模式是一样的，
只有当模型中存在dropout和batchnorm的时候才有区别。